Task 1: Handling Missing Values (Titanic Dataset)

●	Task: Identify missing values, decide whether to fill (mean/median/mode) or drop, and justify your choice.


In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv('Titanic.csv')

print("Initial DataFrame:",df.shape)
df.head()

Initial DataFrame: (418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [2]:
missing_summary = df.isnull().sum().sort_values(ascending=False)
missing_percent = (df.isnull().mean() * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Values': missing_summary,
    'Percentage': missing_percent
})

print("Missing Value Report:")
missing_df

Missing Value Report:


,Missing Values,Percentage
Age,86,20.57
Cabin,327,78.23
Embarked,0,0.00
Fare,1,0.24
Name,0,0.00
Parch,0,0.00
PassengerId,0,0.00
Pclass,0,0.00
Sex,0,0.00
SibSp,0,0.00


In [3]:
threshold = 0.4  
cols_to_drop = missing_df[missing_df['Percentage'] > (threshold*100)].index
df = df.drop(columns=cols_to_drop)
print("Dropped Columns:", cols_to_drop.tolist())

df['Age'] = df['Age'].fillna(df['Age'].median())

df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

print("After Cleaning - Missing Values Check:")
df.isnull().sum()

Dropped Columns: ['Cabin']
After Cleaning - Missing Values Check:


PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Embarked       0
dtype: int64

In [4]:
df.to_csv(r"C:\Users\veerk\OneDrive\Desktop\ARCAP\Arcap_Reits\DS\Task1\cleaned\titanic_cleaned.csv", index=False)
print(" Cleaned Titanic dataset saved as 'titanic_cleaned.csv'")

 Cleaned Titanic dataset saved as 'titanic_cleaned.csv'


Explanation
1.Cabin column dropped because it has >70% missing values → too sparse.
2.Age filled with median (robust against outliers).

3.Embarked filled with mode (categorical).

4.Other columns have no significant missing data.

Task 02 Removing Duplicates (Netflix Dataset)

In [13]:
import pandas as pd

pd.set_option('display.max_columns', None)

df_netflix = pd.read_csv("netflix.csv")
print("Initial Netflix DataFrame:", df_netflix.shape)

df.head()

Initial Netflix DataFrame: (6104, 12)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [23]:
print(f"\nTotal duplicate rows: {df.duplicated().sum()}")


Total duplicate rows: 266


In [21]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('netflix.csv')

# Display basic information about the dataset
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

# Check for duplicate rows
print(f"\nTotal duplicate rows: {df.duplicated().sum()}")

# Check for duplicates based on specific columns (common in Netflix data)
# Common columns to check for duplicates: title, show_id, or combination of title and release_year
if 'show_id' in df.columns:
    print(f"Duplicate show_id: {df['show_id'].duplicated().sum()}")
if 'title' in df.columns:
    print(f"Duplicate titles: {df['title'].duplicated().sum()}")

# If you want to see the actual duplicate rows
duplicate_rows = df[df.duplicated(keep=False)]  # keep=False marks all duplicates
if not duplicate_rows.empty:
    print("\nDuplicate rows:")
    print(duplicate_rows.sort_values(by=df.columns[0]))

# Method 1: Remove all duplicate rows (keeping first occurrence)
df_cleaned = df.drop_duplicates(keep='first')

# Method 2: Remove duplicates based on specific columns
# For example, if you want to remove duplicates based on title
if 'title' in df.columns:
    df_cleaned_title = df.drop_duplicates(subset=['title'], keep='first')

# Method 3: Remove duplicates based on multiple columns
# For example, title and release_year to handle same title from different years
if all(col in df.columns for col in ['title', 'release_year']):
    df_cleaned_multi = df.drop_duplicates(subset=['title', 'release_year'], keep='first')

# Display results after cleaning
print(f"\nShape after removing all duplicates: {df_cleaned.shape}")
print(f"Rows removed: {len(df) - len(df_cleaned)}")

# If you used specific column cleaning
if 'title' in df.columns:
    print(f"Shape after removing title duplicates: {df_cleaned_title.shape}")

# Save the cleaned dataset
df_cleaned.to_csv('netflix_cleaned.csv', index=False)
print("\nCleaned dataset saved as 'netflix_cleaned.csv'")

# Additional analysis: Check for near-duplicates (similar titles)
def find_similar_titles(df, title_column='title'):
    """Find potentially similar titles that might be duplicates"""
    from difflib import SequenceMatcher
    
    titles = df[title_column].dropna().unique()
    similar_pairs = []
    
    for i in range(len(titles)):
        for j in range(i+1, len(titles)):
            similarity = SequenceMatcher(None, titles[i].lower(), titles[j].lower()).ratio()
            if similarity > 0.8:  # Adjust threshold as needed
                similar_pairs.append((titles[i], titles[j], similarity))
    
    return similar_pairs

# Optional: Check for similar titles (might be computationally intensive for large datasets)
if 'title' in df.columns and len(df) < 1000:  # Only run for smaller datasets
    similar_titles = find_similar_titles(df)
    if similar_titles:
        print("\nPotentially similar titles (manual review needed):")
        for title1, title2, similarity in similar_titles:
            print(f"Similarity: {similarity:.2f} - '{title1}' vs '{title2}'")

# Summary report
print("\n" + "="*50)
print("DUPLICATE CLEANING SUMMARY")
print("="*50)
print(f"Original dataset shape: {df.shape}")
print(f"Cleaned dataset shape: {df_cleaned.shape}")
print(f"Total duplicate rows removed: {len(df) - len(df_cleaned)}")
print(f"Percentage reduction: {(len(df) - len(df_cleaned))/len(df)*100:.2f}%")

# Check for missing values in cleaned data
print("\nMissing values in cleaned dataset:")
print(df_cleaned.isnull().sum())

Dataset shape: (6104, 12)

First few rows:
    show_id                                title                    director  \
0  81193313                            Chocolate                         NaN   
1  81197050  Guatemala: Heart of the Mayan World  Luis Ara, Ignacio Jaunsolo   
2  81213894                      The Zoya Factor             Abhishek Sharma   
3  81082007                            Atlantics                   Mati Diop   
4  80213643                      Chip and Potato                         NaN   

                                                cast  \
0  Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang ...   
1                                  Christian Morales   
2  Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, ...   
3  Mama Sane, Amadou Mbow, Ibrahima Traore, Nicol...   
4  Abigail Oliver, Andrea Libman, Briana Buckmast...   

                    country         date_added  release_year rating  \
0               South Korea  November 30, 2019          2019  TV-14 